# Çalışmanın Özeti

Bu çalışmada, biyomekanik verilerine göre hastaların sınıflandırması yapılacaktır. Çalışmanın özeti şöyledir:

-Veri setinin yüklenmesi

-Gerekiyorsa ön işlemlerin yapılması

-Veri setinin görselleştirilmesi

-Ölçeklendirme yapılması

-Yardımcı fonksiyonlarla başarı oranı



In [ ]:
#Gerekli Kütüphanelerin Yüklenmesi Yapılıyor
import numpy as np # linear algebra
import pandas as pd # Veri işleme
#Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns    # görselleştirme aracı        
#Makine Öğrenmesi Gereksimleri
# Ön işleme için
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Model seçimi için
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
# Makine öğrenmesi modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
#Sistem Kütüphaneleri
import os
import warnings
print(os.listdir("../input/biomechanical-features-of-orthopedic-patients"))

# Veriler, iki farklı biçimde sınıflandırma görevi için düzenlenmiştir.

column_2C_weka.csv (iki sınıf etiketli dosya):

İkinci görev için Disk Hernia ve Spondylolisthesis kategorileri 'anormal' olarak etiketlenerek tek bir kategoride birleştirilmiştir. Böylece, hastalar, iki kategoriden birine ait sınıfla etiketlendirilmiştir; Normal (100 hasta) veya Anormal (210 hasta).

Veri Setindeki Özellikler Hakkında

Veri setindeki her hasta(örnek, satır); pelvis ve lomber omurganın (her biri bir sütundur) şekli ve oryantasyonundan türetilen altı biyomekanik özellikle temsil edilir:

pelvik insidans

pelvik eğim

lomber lordoz açısı

sakral eğim

pelvik yarıçap

spondilolistezis derecesi


In [ ]:
#İki sınıf içeren veri seti
veriseti=pd.read_csv("../input/biomechanical-features-of-orthopedic-patients/column_2C_weka.csv")
veriseti.head() #ilk beş satırı listelenecek.

In [ ]:
veriseti.tail() #son beş satırı listelenecek.

In [ ]:
veriseti.describe() #Veri Setinin dağılımını ve şeklini özetlemek için kulalnılır.


*Veri seti 310 örnekten oluşmaktadır.

*Tüm özellikler sayısal ve eksiksiz olduğu için, verilerin sayıya dönüştürülmesi ve eksik bilgilerin doldurulması gibi ön işlemlere gerek yoktur.

*Özellik sütünlarında yer alan özelliklerin değer aralıkları birbirinden çok farklı olduğu için ölçeklendirme yapılmalıdır. Örneğin; pelvic_radius özelliği 100'den büyük değerler alabilirken degree_spondylolisthesis 5'e kadar değerler alabilmektedir.

In [ ]:
veriseti.info() #veri setinin kısa bir özetini almak için kullanılır.

In [ ]:
veriseti.shape # Satır ve Sütun sayısı

# **Veri Setlerinin Görselleştirilmesi**
Veri setinin görselleştirmesine geçebiliriz. Makine öğremesi modellerine hazır hale gelebilmesi için, veri setindeki değerlerin sayısal bir değer olması gerekir(Not: Görselleştirme, bazı grafikler için sayısal değerler olmadan da yapılabilir). Sayısal değerler iki farlı nitelikte olabilir; sürekli ve ayrık değişkenler.

Sürekli değişken: Sonsuz sayıda olası değere sahip bir değişkendir. Başka bir deyişle; değişken için herhangi bir değer mümkündür.

Ayrık değişken: Yalnızca belirli bir sayıda değer alabilen bir değişkendir. Başka bir deyişle; sonsuz sayıda değere sahip değildir.

In [ ]:
def discrete_univariate(dataset, discrete_feature):
    fig, axarr=plt.subplots(nrows=1,ncols=2, figsize=(8,5))
      
    dataset[discrete_feature].value_counts().plot(kind="bar",ax=axarr[0])
    dataset[discrete_feature].value_counts().plot.pie(autopct="%1.1f%%",ax=axarr[1])
        
    plt.tight_layout()
    plt.show()

In [ ]:
discrete_univariate(dataset=veriseti, discrete_feature="class")

Yukarıdaki şekilde iki sınıfın dağılımı bar ve daire grafikleriyle gösterilmiştir. Bar grafiği sınıfların dağılımlarını sayısal olarak yorumlamaya imkan verirken daire grafiği yüzdelik olarak yorumlamaya daha elverişlidir.

# Tek Sürekli Değişken Dağılımı(Univariate Visualization) 

Tek sürekli değişken dağılımını göstermek için kullanılabilecek en iyi grafik türü histogramdır. Histogram, bar grafiğin sürekli değişkenler için uyarlanmış hali gibi düşünülebilir. Histogram grafiğinde sürekli değişkenler istenilen sayıda sütünla temsil edilir. Sütünların boyları, belirli aralıktaki değerlerin sıklığına göre elde edilir.

Tek sürekli değişimin dağılımını gösteren bir diğer grafik türü Kernel Density Estimation(KDE) grafiğidir. KDE sürekli değişkenin dağılımını tek bir çizgiyle gösterir. Histogram sütünlarının üst kısımlarının bir çizgiyle birleştirilmesiyle KDE grafiği elde edilir. Genelde histogram ve KDE çizimleri tek bir grafikte gösterilir.

In [ ]:
def continuous_univariate(dataset, continuos_feature):
    fig, ax=plt.subplots(nrows=2,ncols=3, figsize=(12,8))

    ax=ax.flatten()

    "pandas çizim fonksiyonları kullanılıyor"
    veriseti[continuos_feature].plot.hist(density=True,ax=ax[0])
    veriseti[continuos_feature].plot.kde(ax=ax[1])
    veriseti[continuos_feature].plot.hist(density=True,ax=ax[2])
    veriseti[continuos_feature].plot.kde(ax=ax[2])

    "Seaborn çizim fonksiyonları kullanılıyor"
    sns.distplot(a=veriseti[continuos_feature], kde=False, ax=ax[3])
    sns.distplot(a=veriseti[continuos_feature], hist=False, ax=ax[4])
    sns.distplot(a=veriseti[continuos_feature], ax=ax[5]) 

In [ ]:
continuous_univariate(dataset=veriseti, continuos_feature="pelvic_incidence")

Yukarıdaki şekilde pelvic_incidence özelliğinin dağılımı histogram ve KDE grafikleriyle gösterilmektedir. Birinci satırdaki grafikler pandas fonksiyonlarıyla, ikinci satırdaki grafikler seaborn fonksiyonlarıyla elde edilmiştir.

pelvic_incidence değişkenin dağılımı normal(Gaussian) dağılıma yakın bir yapıdadır. Değişken dağılımı ortalamanın sağında biraz daha fazladır. Çoğu makine öğrenmesi modeli normal dağılıma sahip veri setlerinde daha iyi sonuç verir.

Tüm değişkenlerin histogramın grafiğini pandas DataFrame nesnesinde yer alan hist() kolayca elde edilebilir.

In [ ]:
veriseti.hist(bins=10, density=True, figsize=(15,8))
plt.show()

Tüm sürekli değişkenleri histogram ve kde sağılımlarını seaborn kullanarak aşağıdaki gibi elde edebiliriz.

In [ ]:
fig, ax=plt.subplots(nrows=2, ncols=3,figsize=(12,8))
ax=ax.flatten()
col_names=veriseti.drop('class', axis=1).columns.values

for i,col_name in enumerate(col_names):
    sns.distplot(a=veriseti[col_name], ax=ax[i])

Yukarıdaki tabloda tüm sürekli değişkenlerin dağılımı yer almaktadır.

# Sınıf Dağılımlar ve Özellik Değerlerinin Sınıflar Üzerindeki Dağılımı 

'class' değişkeni ayrık bir değer olduğu için diğer değişkenlerle olan ilişkisi kategorik verilerin görselleştirmesinde kullanılan çizim türleri tercih edilecektir. Seaborn kütüphanesinde kategorik değerler için kullanılan çizim türler şunlardır: stripplot(), swarmplot(), boxplot(), violinplot(), boxenplot() pointplot(), barplot(), countplot().

Öncelikle sınıf dağılımlarını öğrenmek için countplot() grafik türünü kullanalım. Daha sonra çıktı değişkeniyle herbir girdi değişkeni arasındaki ilişkiyi swarmplot ile gösterelim

In [ ]:
def plot_categorical(dataset, categorical_feature, rows, cols):
    fig, axarr=plt.subplots(nrows=2,ncols=4, figsize=(15,11))
    features=dataset.columns.values[:-1]
    
    counter=0
    #sns.countplot(x=categorical_feature, data=dataset, ax=axarr[0,0])
    dataset['class'].value_counts().plot.bar(ax=axarr[0,0])
    dataset['class'].value_counts().plot.pie(autopct="%1.1f%%",ax=axarr[0,1])
    for i in range(rows):
        for j in range(cols):
            feature=features[counter]
            if (i==0 and j==0) or (i==0 and j==1):
                continue
            else:
                sns.swarmplot(x=categorical_feature,y=feature,
                             
                            data=dataset, 
                            ax=axarr[i, j])
            counter=counter+1
            if counter>=len(features):
                break
    
    plt.tight_layout() 
    plt.show()

In [ ]:
plot_categorical(dataset=veriseti, categorical_feature="class", rows=2, cols=4)

# Özelliklerin Birbirleriyle Olan İlişkisinin Gösterilmesi

Tüm değişkenlerin birbirleri arasındaki ilişkiyi pairplot ve korelasyon matrisyle gözlemleyebiliriz.

pairplot tüm değişkenlerin birbirleriyle olan ilişkilerini serpme(scatter) grafiklerle gösterir. iki değişken arasında lineer bir ilişki varsa bu değişkenlerden birini çıkarmak makine öğrenmesi modelinin performansına olumlu etki yapma olasılığı yüksektir.

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren pairplot
sns.pairplot(veriseti, hue="class", markers=["o", "s"])

Korelasyon matrisi değişkenlerin birbiriyle olan ilişkini yönünü ve gücünü belirler. piarplot grafiğinin sunduğu ilişkiyi daha açık ve sayısal değerlerle sunar. Korelasyon matrisi veri setini bir bütün olarak yorumlamak için en iyi grafik türüdür.

In [ ]:
corr=veriseti.corr()
fig, ax=plt.subplots(1,1,figsize=(12,8))
sns.heatmap(corr,annot=True, linewidth=.5, ax=ax)

Yardımcı Fonksiyon: draw_multivarient_plot(dataset, rows, cols, plot_type)

Veri setindeki özelliklerin, sınıf ile ilişkilerini yorumlayabilmek için;

Keman Grafiği (Violin Plot)
Kutu Grafiği (Box Plot)
Nokta Grafiği (Point Plot)
Bar Grafiği (Bar Plot) grafkilerini kullanabiliriz.

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren farklı grafikler;
#boxplot, violinplot, pointplot, barplot
def draw_multivarient_plot(dataset, rows, cols, plot_type):
    
    assert plot_type in ['violin', "box", "point", "bar"],"We dont have such as plot type:{}".format(plot_type)
    column_names=dataset.columns.values
    number_of_column=len(column_names)
    fig, axarr=plt.subplots(rows,cols, figsize=(22,16))
    
    counter=0
    for i in range(rows):
        for j in range(cols):
            if 'violin' in plot_type:
                sns.violinplot(x='class', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='class', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='class',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='class',y=column_names[counter],data=dataset, ax=axarr[i][j])
                
            counter+=1
            if counter==(number_of_column-1,):
                break

In [ ]:
draw_multivarient_plot(dataset=veriseti, rows=2, cols=3,plot_type="violin") #KemanGrafiği

In [ ]:
draw_multivarient_plot(dataset=veriseti, rows=2, cols=3,plot_type="point") #NoktaGrafiği

In [ ]:
draw_multivarient_plot(dataset=veriseti, rows=2, cols=3,plot_type="box") #KutuGrafiği

In [ ]:
draw_multivarient_plot(dataset=veriseti, rows=2, cols=3,plot_type="bar") #BarGrafiği

# Sınıflandırma Performansı
# Yardımcı Fonksiyonlar 

get_models(): Kullanılacak makine öğrenmesi algoritmalarına geri dönen fonksiyon

get_X_and_y(): Veri setini 'veri(X)' ve 'çıktı(y)' olarak ayrıştırır.

accuracy_scores_for_various_ml_models(): Kullanılan makine öğrenmesi algoritmlarının başarı oranlarına geri döner


In [ ]:
def get_models():
    models=[]
    models.append(("LR",LogisticRegression()))
    models.append(("LDA", LinearDiscriminantAnalysis()))
    models.append(("NB",GaussianNB()))
    models.append(("KNN",KNeighborsClassifier(n_neighbors = 3)))
    models.append(("DTC",DecisionTreeClassifier()))
    models.append(("SVM rbf",SVC()))
    models.append(("SVM linear",SVC(kernel='linear')))
    
    return models

In [ ]:
def get_X_and_y(dataset, target_name):
    X=dataset.drop(target_name, axis=1)
    y=dataset[target_name]

    X=StandardScaler().fit_transform(X)
    
    labelEncoder=LabelEncoder()
    y=labelEncoder.fit_transform(y)
    
    return X, y

def accuracy_scores_for_various_ml_models(X, y):
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
    models=get_models()
    for name, model in models:
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        score=model.score(X_test, y_test)
        
        print("{} accuracy:{:.2f}".format(name, score))
        print("Karmaşıklık Matrisi\n", confusion_matrix(y_test, pred))
        print("\n Sınıflandırma Raporu\n\n", classification_report(y_test, pred))

# BAŞARI SONUÇLARI

In [ ]:
print("Veri Seti İçin Başarı Oranları")
X, y=get_X_and_y(dataset=veriseti, target_name='class')
accuracy_scores_for_various_ml_models(X, y)